In [1]:
import git
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn_genetic import GAFeatureSelectionCV
from sklearn_genetic.space import Categorical, Integer, Continuous

In [2]:
# Testing whether using data_consol.csv helps anything. If so, probably indicates an error in reading in or joining the separate CSVs before
repo = git.Repo('.', search_parent_directories = True)
root = repo.working_tree_dir

data_consol = pd.read_csv(root + '//data/data_consol.csv')

In [3]:
X = data_consol.filter(regex="^[0-9]+$")
bact = data_consol['pcr_bact_log']

# Note: do NOT scale X and y before splitting, since that is a data leak. Instead, use the pipeline to scale both Xs, and separately scale the y for custom scoring like RMSE.
X_train, X_test, bact_train_unscaled, bact_test_unscaled = train_test_split(X.to_numpy(), bact.to_numpy(), train_size=0.8, random_state=0)

# Reshaping necessary for the y scaling step
bact_train_unscaled = bact_train_unscaled.reshape(-1,1)
bact_test_unscaled = bact_test_unscaled.reshape(-1,1)

bact_scaler = StandardScaler()
bact_train = bact_scaler.fit_transform(bact_train_unscaled).reshape(-1,1)
bact_test = bact_scaler.transform(bact_test_unscaled).reshape(-1,1)

# 10-fold CV; random state 0
cv_5_0 = KFold(n_splits=5, shuffle=True, random_state=0)

In [4]:
model = ElasticNet(fit_intercept=False, warm_start=True, random_state=0, selection='random', max_iter=4000)


Now let's set up the feature selector object

In [22]:
# Define the genetic algorithm feature selector
selector = GAFeatureSelectionCV(
    estimator=model,
    cv=cv_5_0,  # Cross-validation folds
    scoring="neg_root_mean_squared_error",  # Fitness function (maximize accuracy)
    population_size=20,  # Number of individuals in the population
    generations=50,  # Number of generations
    n_jobs=-1,  # Use all available CPU cores
    verbose=True,  # Print progress
    max_features = 16
)

In [23]:
pipe = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("features", selector),
        ("elastic_net", model)
    ], 
    memory = root+'\\cache',
    verbose=True
)

REGULARIZATION = np.logspace(-5, 0, 8)
MIXTURE = np.linspace(0.001, 1, 8)
PARAM_GRID = [
    {
        "elastic_net__alpha": REGULARIZATION,
        "elastic_net__l1_ratio": MIXTURE
    }
]

grid = GridSearchCV(estimator=pipe, param_grid=PARAM_GRID, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=cv_5_0, error_score='raise')
grid.fit(X_train, bact_train)

C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	-40000.6	48989.3    	-0.99933   	-100000    
1  	40    	-35000.6	47696.5    	-0.99933   	-100000    
2  	40    	-25000.7	43300.8    	-0.99933   	-100000    
3  	40    	-20000.8	39999.6    	-0.99933   	-100000    
4  	40    	-20000.8	39999.6    	-0.99933   	-100000    
5  	40    	-30000.7	45825.3    	-0.99933   	-100000    
6  	40    	-30000.7	45825.3    	-0.99933   	-100000    
7  	40    	-55000.4	49748.9    	-0.99933   	-100000    
8  	40    	-70000.3	45825.3    	-0.99933   	-100000    
9  	40    	-65000.3	47696.5    	-0.99933   	-100000    
10 	40    	-65000.3	47696.5    	-0.99933   	-100000    
11 	40    	-80000.2	39999.6    	-0.99933   	-100000    
12 	40    	-85000.1	35706.8    	-0.99933   	-100000    
13 	40    	-80000.2	39999.6    	-0.99933   	-100000    
14 	40    	-75000.2	43300.8    	-0.99933   	-100000    
15 	40    	-65000.3	47696.5    	-0.99933   	-100000    
16 	40    	-50000.5	49999.5    	-0.99933   	-100

C:\Users\joshua.waldbieser\.conda\envs\rkngh_stress\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.433e+02, tolerance: 3.180e-02
  model = cd_fast.enet_coordinate_descent(


GridSearchCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
             error_score='raise',
             estimator=Pipeline(memory='C:\\Users\\joshua.waldbieser\\OneDrive '
                                       '- '
                                       'USDA\\root_knot_nematode_greenhouse\\paper_2\\DirtSpectra\\cache',
                                steps=[('scaler', StandardScaler()),
                                       ('features',
                                        GAFeatureSelectionCV(cv=KFold(n_splits=5, random_state=0, shuffle=True),
                                                             estimator=Elast...
                                                   warm_start=True))],
                                verbose=True),
             n_jobs=-1,
             param_grid=[{'elastic_net__alpha': array([1.00000000e-05, 5.17947468e-05, 2.68269580e-04, 1.38949549e-03,
       7.19685673e-03, 3.72759372e-02, 1.93069773e-01, 1.00000000e+00]),
                          'elastic_net__l1_ratio': array([0.001     , 0.14371429, 0.28642857, 0.42914286, 0.57185714,
       0.71457143, 0.85728571, 1.        ])}],
             scoring='neg_root_mean_squared_error')

In [24]:
print('Training RMSE:', round(abs(grid.score(X_train, bact_train)), 3))
print('Testing RMSE:', round(abs(grid.score(X_test, bact_test)), 3))

# Inverse-transforming the preds to get back to original scale.
# Used for comparison with R results
preds_unscaled = bact_scaler.inverse_transform(grid.predict(X_test).reshape(-1,1))
print('Testing RMSE, unscaled:', round(root_mean_squared_error(preds_unscaled, bact_test_unscaled), 3))

Training RMSE: 0.95
Testing RMSE: 0.988
Testing RMSE, unscaled: 0.436


In [27]:
best_pipe = grid.best_estimator_
selector = best_pipe.named_steps['features'] # .best_features_

# Check if the selector has the 'best_features_' attribute
if hasattr(selector, "best_features_"):
    # Get the mask of selected features (True for selected, False for not selected)
    selected_features_mask = selector.best_features_

    # Get the feature names (if available)
    feature_names = X_train.columns if hasattr(X_train, "columns") else [f"Feature_{i}" for i in range(X_train.shape[1])]
    selected_feature_names = [name for name, selected in zip(feature_names, selected_features_mask) if selected]

    # print("Selected Features:", selected_feature_names)
else:
    print("The attribute 'best_features_' is not available.")


In [28]:
# Looking at the wavelengths that were chosen, but in a slightly different display format
feats = best_pipe.named_steps['features'].best_features_
wvs = np.arange(350,2501)
print(wvs[feats])

[ 482  631  867  929 1088 1376 1663 1666 2343 2429]


*Temp results tracker*

max_features=16 ->
Training RMSE: 0.949
Testing RMSE: 0.984
Testing RMSE, unscaled: 0.434
[ 426  712 1295 1446 1878 2073 2423]

max_features=None ->
Training RMSE: 0.546
Testing RMSE: 0.771
Testing RMSE, unscaled: 0.34
967 features chosen